In [ ]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/siddukasam28/PracticeDataScienceProject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "siddukasam28"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "63c4cbbeb538d23123b3424e131fc50146474157"

In [2]:
import os
%pwd

'c:\\Users\\Siddu\\Downloads\\New folder\\MLOps\\PracticeDataScienceProject\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\Siddu\\Downloads\\New folder\\MLOps\\PracticeDataScienceProject'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir:Path
    test_data_path:Path
    model_path:Path
    all_params:dict
    metric_file_name:Path
    target_column:str
    mlflow_uri:str

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories,save_json

In [10]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/siddukasam28/PracticeDataScienceProject.mlflow"
        )

        return model_evaluation_config

In [11]:
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [22]:
class ModelEvaluation:
    def __init__(self,config=ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        return rmse,mae,r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)
            (rmse,mae,r2) = self.eval_metrics(test_y,predicted_qualities)

            scores = {"rmse":rmse,"mae":mae,"r2":r2}
            save_json(path = Path(self.config.metric_file_name),data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("r2",r2)

            os.makedirs("model",exist_ok=True)
            joblib.dump(model,"model/model.pkl")

            mlflow.log_artifacts("model",artifact_path="model")

In [21]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2025-11-29 16:13:36,213: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-29 16:13:36,215: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-29 16:13:36,218: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-11-29 16:13:36,220: INFO: common: created directory at: artifacts]
[2025-11-29 16:13:36,221: INFO: common: created directory at: artifacts/model_evaluation]
[2025-11-29 16:13:36,591: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
🏃 View run stylish-bird-417 at: https://dagshub.com/siddukasam28/PracticeDataScienceProject.mlflow/#/experiments/0/runs/71936bcb66944505a24c6763dfbd4eaf
🧪 View experiment at: https://dagshub.com/siddukasam28/PracticeDataScienceProject.mlflow/#/experiments/0
